# Segmenting and Clustering

### Import necessary libraries

In [2]:
!pip install BeautifulSoup4
!pip install requests

     |████████████████████████████████| 122kB 6.4MB/s eta 0:00:01


In [6]:
!pip install lxml

     |████████████████████████████████| 5.5MB 5.2MB/s eta 0:00:01


In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Import data

In [15]:
# get html from wiki page and create soup object
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
link_text = requests.get(link).text
soup = BeautifulSoup(link_text, 'html5lib')

In [34]:
# transform data into Pandas DataFrame
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)

canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning

In [35]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood 
canada_df.rename(columns={"Postal Code": "PostalCode", "B": "c"}, inplace=True)
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [36]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [38]:
# More than one neighborhood can exist in one postal code area. Combine them into one row with the neighborhoods separated with a comma.
canada_df['Neighborhood'] = canada_df.groupby('PostalCode')['Neighborhood'].transform(lambda neigh: ', '.join(neigh))

canada_df = canada_df.drop_duplicates()
    
canada_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [39]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
canada_df['Neighborhood'].replace('Not assigned', canada_df['Borough'],inplace=True)
canada_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
# reset index
canada_df.reset_index(drop = True, inplace = True)
canada_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [30]:
canada_df.shape

(103, 3)

### Add geo data

In [43]:
# get the data
!wget -q -O "geo_data.csv" http://cocl.us/Geospatial_data
geo_data = pd.read_csv('geo_data.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
# merge two dataframes
canada_df_temp = canada_df.set_index('PostalCode')
geo_data_temp = geo_data.set_index('Postal Code')
new_df = pd.concat([canada_df_temp, geo_data_temp], axis=1, join='inner')

new_df.index.name = 'PostalCode'
new_df.reset_index(inplace=True)

new_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [45]:
new_df.shape

(103, 5)